# Helper-Funktionen

In [1]:
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

# Get Content

In [96]:
def getContent(entityId):
    return json.loads(query("""
        select entity_revision_field.value from entity
        join entity_revision on entity_revision.id = entity.current_revision_id
        join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity.id = %s and entity_revision_field.field = "content"
    """ % entityId)[0][0])

getContent(1655)

[[{'col': 12,
   'content': 'Die Wertemenge (oder Bildmenge) einer [Funktion](/1583) ist die Menge aller mÃ¶glichen Funktionswerte, die herauskommen kÃ¶nnen, wenn man alle Zahlen aus der [Definitionsmenge](/1575) in die Funktion einsetzt.'},
  {'col': 12,
   'content': '\n\n![Geogebra File: https://assets.serlo.org/legacy/2169_wEc78ao5uo.xml](https://assets.serlo.org/legacy/2170_gs696gRTIZ.png)'}],
 [{'col': 24,
   'content': '\n\n\n## Unterschied zwischen Wertemenge und Zielmenge\n\nIn der Wertemenge befinden sich nur die Werte, dieÂ\xa0wirklich von der Funktion angenommen werden.\n\nDie Zielmenge ist eine Menge, in der die Wertemenge enthalten ist, es aber auch noch weitere Elemente gibt. So ist z. B. eine Zielmenge der Funktion %%f(x)=x^2%% die Menge der  [reellen Zahlen](/1693) %%\\mathbb{R}%% . Die eigentliche Wertemenge ist aber %%\\mathbb{R}_0^+%% (alle positiven reellen Zahlen und die Null), da keine negativen Zahlen von der Funktion angenommen werden.\n\nÂ\xa0Â\xa0Â\xa0\n\n\n#

# Find Injections

In [109]:
import json
import re

from IPython.display import display, Markdown

def getInjectionIds(content, e=0):
    if "plugin" in content:
        return getInjectionIdsEdtrIo(content, e)
    else:
        return getInjectionIdsLegacy(content)
    
def getInjectionIdsLegacy(content):
    regex = re.compile(r">\[[^\]]+\]\(\/(\d+)\)")
    
    for row in content:
        for column in row:
            for match in regex.finditer(column["content"]):
                yield int(match.group(1))

def getInjectionIdsEdtrIo(content, e):
    if isinstance(content, list):
        for child in content:
            yield from getInjectionIdsEdtrIo(child, e)
    if isinstance(content, dict):
        if "plugin" in content and content["plugin"] == "injection":
            try:
                yield int(content["state"][1:])
            except ValueError:
                print("Wrong state in id %s: %s" % (e, content["state"]))
        else:
            for child in content.values():
                yield from getInjectionIdsEdtrIo(child, e)


# {'plugin': 'injection', 'state': '/5679'}

display(Markdown("Injected entities from 1549:"))
display(list(getInjectionIds(getContent(1549))))

display(Markdown("Injected entities from 2153:"))
display(list(getInjectionIds(getContent(2153))))

Injected entities from 1549:

[5679, 5683, 5673, 5761]

Injected entities from 2153:

[9775, 9779, 9783, 32321]

# List of all entity ids

In [90]:
entityIds = querySingleton("""
    select entity.id from entity
    join uuid on uuid.id = entity.id
    join type on type.id = entity.type_id
    where uuid.trashed = 0
        and type.name != "comment"
        and type.name != "video"
        and type.name != "course"
        and type.name != "input-number-exact-match-challenge"
        and type.name != "input-expression-equal-match-challenge"
        and type.name != "input-string-normalized-match-challenge"
        and type.name != "multiple-choice-right-answer"
        and type.name != "multiple-choice-wrong-answer"
        and type.name != "single-choice-right-answer"
        and type.name != "single-choice-wrong-answer"
        and entity.current_revision_id is not null
""")

len(entityIds)

23299

# Retrieve contents

In [91]:
from json import JSONDecodeError

contents = dict()

for entityId in entityIds:
    try:
        contents[entityId] = getContent(entityId)
    except JSONDecodeError:
        print("Error with id: %s" % entityId)
    except IndexError:
        print("Error with id: %s" % entityId)

Error with id: 14573
Error with id: 41722
Error with id: 41729


# Compute injections

In [115]:
result = dict()

for entityId in entityIds:
    if entityId in contents:
        try:
            for injectedId in getInjectionIds(contents[entityId], entityId):
                if injectedId not in result:
                    result[injectedId] = []
                
                result[injectedId].append(entityId)
        except TypeError:
            print(entityId)
            print(contents[entityId])

len(result)

Wrong state in id 11995: https://assets.serlo.org/legacy/541c0f4abafce_a6939af75e1532d9027d2bc2e7a8d6721f73beeb.jpg
Wrong state in id 29130: https://assets.serlo.org/legacy/54081f134e254_a6939af75e1532d9027d2bc2e7a8d6721f73beeb.jpg
Wrong state in id 34159: 1
Wrong state in id 90405: /ref/11391
Wrong state in id 90405: /ref/9181
Wrong state in id 90405: /ref/6999
Wrong state in id 90405: /ref/54210
Wrong state in id 90405: /ref/62918
Wrong state in id 90405: /ref/60321
Wrong state in id 90405: /ref/58828
Wrong state in id 137883: ggt\awsn27zf


1513

# Auflistung aller Injektions

In [122]:
items = list(result.items())
items.sort(key=lambda x: x[0])

for entityId, injectedIds in items:
    display(Markdown("### Id:%s wird eingebunden in:" % entityId))
    display(Markdown("\n".join([
        "* https://serlo.org/%s" % x for x in injectedIds
    ])))

### Id:107 wird eingebunden in:

* https://serlo.org/144838

### Id:165 wird eingebunden in:

* https://serlo.org/2193

### Id:201 wird eingebunden in:

...

* https://serlo.org/192432

### Id:192500 wird eingebunden in:

* https://serlo.org/175299

### Id:192545 wird eingebunden in:

* https://serlo.org/192531